<a href="https://colab.research.google.com/github/ahaque12/fiddler-even-odds/blob/main/Fiddler_on_the_Proof_Even_the_Odds.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Fiddler on the Proof - Even the Odds

https://thefiddler.substack.com/p/can-you-even-the-odds

In [1]:
import random

def game(generator, p):
  """Simulate a game with two players. Each player takes turns, the next
  item in generator says if it is player 0's turn or player 1's turn.
  When it is their turn they have p probability of winning. The game ends as soon
  as someone wins. Simulate the game.
  """
  for i in generator:
    if random.random() < p:
      return i

  return None

def simulate_game(n, gen_func, p):
  """Simulate game n times and return % times player 0 wins."""
  wins = 0
  for i in range(n):
    if game(gen_func(), p) == 0:
      wins += 1
  return wins / n

def derive_winning_incr(n, generator, p):
  """Iterate through n items from generator incrementally adding
  the probability of winning and returning the est probability of winning (x) and
  the probability of the game having not ended (sigma). The true probability of
  winning is in (x, x + sigma).
  """
  total = 0
  delay = 1
  for i in range(n):
    result = next(generator)
    if result == 0:
      total += delay*p

    delay *= 1 - p
  return total, delay

In [2]:
def alternate():
  """Generator that alternates between 0 and 1."""
  while True:
    yield 0
    yield 1

def snake():
  """Generator that uses snake method."""
  while True:
    yield 0
    yield 1
    yield 1
    yield 0

def thue_morse_sequence():
    n = 0
    while True:
        # Count the number of 1s in the binary representation of n
        # The next player is determined by the parity of this count
        count_of_ones = bin(n).count('1')
        next_player = count_of_ones % 2
        yield next_player
        n += 1

games = lambda : [("Alternate",alternate()),
          ("Snake",snake()),
          ("Thue Morse",thue_morse_sequence())]

In [3]:
import itertools
def first_n_items(generator, n):
  """Generator that returns first n items of generator."""
  return list(itertools.islice(generator, n))

In [4]:
for name, gen in games():
  print(name, first_n_items(gen, 16))

Alternate [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1]
Snake [0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0]
Thue Morse [0, 1, 1, 0, 1, 0, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0]


In [5]:
for name, gen in games():
  print(name, derive_winning_incr(1000, gen, 1/6))

Alternate (0.5454545454545457, 6.588005489477521e-80)
Snake (0.5081967213114751, 6.588005489477521e-80)
Thue Morse (0.5015903392042689, 6.588005489477521e-80)


In [6]:
import numpy as np

def conf(N, generator, p):
  """Generate 95% confidence interval for monte carlo simulation in simulate_game."""
  x = simulate_game(N, generator, p)
  return x, x - 1.96 * np.sqrt(x * (1 - x) / N), x + 1.96 * np.sqrt(x * (1 - x) / N)

In [7]:
conf(1000000, alternate, 1/6), 6/11

((0.546468, 0.5454922413571647, 0.5474437586428352), 0.5454545454545454)

In [8]:
conf(1000000, snake, 1/6), 31/61

((0.50748, 0.5065001096689205, 0.5084598903310796), 0.5081967213114754)

In [9]:
conf(1000000, thue_morse_sequence, 1/6)

(0.501368, 0.500388003667998, 0.5023479963320021)

In [10]:
def min_seq(generator, max=10000):
  """Find the minimum sequence length where the full sequence 0 to n repeats.
  Iterate to 2n items and check if generator[:n] == generator[n:2n]. Once this
  is true end the loop and return n.
  """
  n = 1
  revealed_seq = []
  while True:
    while len(revealed_seq) < 2*n:
      revealed_seq.append(next(generator))
    if revealed_seq[:n] == revealed_seq[n:2*n]:
      return n
    if n == max:
      return None
    n += 1

In [11]:
for name, gen in games():
  print("Minimum sequence length for {} is {}".format(name, min_seq(gen)))

Minimum sequence length for Alternate is 2
Minimum sequence length for Snake is 4
Minimum sequence length for Thue Morse is None
